# Trabajo final data mining

En este trabajo brindaremos un chat bot de respuestas cortas para IM en español. Primero que nada definiremos una respuesta corta como cualquier turno hasta 3 palabras que ocurren al menos n veces. Iremos probando diferentes n a modo de prueba. El texto anterior a la respuesta corta será llamado contexto

## Extracción del corpus

Debido a que no hay un corpus bien definido para este problema utilizaremos el contexto del chat como texto de entrenamiento, por lo que la respuesta corta será la etiqueta.

### Procesamiento

In [4]:
import re
import csv
import pandas as pd
from datetime import datetime, timedelta
from string import punctuation

#### Conociendo los datos "crudos"

In [5]:
with open('whatsapp/chat2.txt') as f:
    raw_data = f.readlines()
print(raw_data[:20])

['3/4/17, 15:52 - Los mensajes y llamadas en este chat ahora están protegidos con cifrado de extremo a extremo. Toca para más información.\n', '3/4/17, 15:52 - Jose: Capaz llego unos minutos más tarde 🙈\n', '3/4/17, 15:52 - Jose: Aviso por si no saliste todavia\n', '3/9/17, 12:07 - Jose: Mati!\n', '3/9/17, 12:07 - Jose: como andás?\n', '3/9/17, 12:07 - Jose: uds tienen idea como se masteriza, normaliza y comprime para la muestra? 😳\n', '3/9/17, 12:07 - Jose: estoy en bolas mal\n', '3/9/17, 12:08 - M: <Archivo omitido>\n', '3/9/17, 12:10 - Jose: <Archivo omitido>\n', '3/9/17, 12:14 - M: <Archivo omitido>\n', '3/9/17, 12:15 - Jose: jajaja bueno, calculo que varios van a hacer eso\n', '3/9/17, 12:16 - Jose: voy a ver si encuentro algo en internet y si no bueno, ya fue\n', '3/9/17, 12:16 - Jose: al final pudiste ver lo que te mandé al face?\n', '3/9/17, 13:03 - M: <Archivo omitido>\n', '3/9/17, 13:18 - Jose: jajaja no el fondo verde ni a palos 😂 puse ahí que era para ver un color que despe

Sacaremos la primera línea que de nada sirve

`6/18/17, 16:43 - Los mensajes y llamadas en este chat ahora están protegidos con cifrado de extremo a extremo. Toca para más información.`

#### Tratando de preprocesar

Para ello spliteo cada línea por la primera ocurrencia de ": ". Ver bien si se puede hacer algo con las líneas que no tengan el mismo formato (por ahora la quitamos). También quitaremos las stops words y reduciremos ocurrencias de jaja a ja (por ahora hacerlo bien básico).

In [65]:
STOP_WORDS = punctuation

def remove_dale(text):
    return re.sub('d[a]+l[e]+', 'dale', text)

def remove_ok(text):
    return re.sub(' (ok.*)+', 'ok', text)

def remove_si(text):
    return re.sub('(s[ií]+)+', 'si', text)

def remove_sw(text):
    return re.sub('{1}{0}{1}'.format(STOP_WORDS, '[]'), '', text).lower()

def remove_ja(text):
    return re.sub('(ja[aj]*)+', 'ja', text)

def clean_text(text):
    return remove_dale(remove_si(remove_ok(remove_ja(remove_sw(text)))))

In [66]:
info_chat = []

for line in raw_data:
    if "-" not in line or ": " not in line:
        continue
    info, text = line.split(": ", 1)
    if '<Archivo omitido>' in text or '<Media omitted>' in text:
        continue
    date, owner = info.split(" - ", 1)
    date, hour = date.split(", ")
    hours, mins = hour.split(":")
    month, day, year = date.split("/")
    cleaning_text = clean_text(text.split('\n')[0])
    info_chat.append([owner, datetime(int('20' + year), int(month), int(day), int(hours), int(mins)), cleaning_text])
info_chat[:10]

[['Jose',
  datetime.datetime(2017, 3, 4, 15, 52),
  'capaz llego unos minutos más tarde 🙈'],
 ['Jose',
  datetime.datetime(2017, 3, 4, 15, 52),
  'aviso por si no saliste todavia'],
 ['Jose', datetime.datetime(2017, 3, 9, 12, 7), 'mati'],
 ['Jose', datetime.datetime(2017, 3, 9, 12, 7), 'como andás'],
 ['Jose',
  datetime.datetime(2017, 3, 9, 12, 7),
  'uds tienen idea como se masteriza normaliza y comprime para la muestra 😳'],
 ['Jose', datetime.datetime(2017, 3, 9, 12, 7), 'estoy en bolas mal'],
 ['Jose',
  datetime.datetime(2017, 3, 9, 12, 15),
  'ja bueno calculo que varios van a hacer eso'],
 ['Jose',
  datetime.datetime(2017, 3, 9, 12, 16),
  'voy a ver si encuentro algo en internet y si no bueno ya fue'],
 ['Jose',
  datetime.datetime(2017, 3, 9, 12, 16),
  'al final pudiste ver lo que te mandé al face'],
 ['Jose',
  datetime.datetime(2017, 3, 9, 13, 18),
  'ja no el fondo verde ni a palos 😂 puse ahí que era para ver un color que despegara']]

Ahora que tenemos los datos un poco más limpios utilizaremos panda para sacar un poco de conclusiones.

In [61]:
df = pd.DataFrame(info_chat, columns=['owner', 'date', 'text'])
df

,owner,date,text
0,Jose,2017-03-04 15:52:00,capaz llego unos minutos más tarde 🙈
1,Jose,2017-03-04 15:52:00,aviso por si no saliste todavia
2,Jose,2017-03-09 12:07:00,mati
3,Jose,2017-03-09 12:07:00,como andás
4,Jose,2017-03-09 12:07:00,uds tienen idea como se masteriza normaliza y ...
5,Jose,2017-03-09 12:07:00,estoy en bolas mal
6,Jose,2017-03-09 12:15:00,ja bueno calculo que varios van a hacer eso
7,Jose,2017-03-09 12:16:00,voy a ver si encuentro algo en internet y si n...
8,Jose,2017-03-09 12:16:00,al final pudiste ver lo que te mandé al face
9,Jose,2017-03-09 13:18:00,ja no el fondo verde ni a palos 😂 puse ahí que...


Ahora agruparemos los contextos con sus respectivas respuestas. Guardaremos el corpus en formato CSV.

In [56]:
MIN = 60
HOUR = 60*MIN
SAME_CONTEXT = timedelta(seconds=24*HOUR)
SAME_TEXT = timedelta(seconds=30*MIN)

SHORT_ANSWER = 2
def is_short(answer):
    """
    is short answer?
    """
    return len(answer.split(' ')) <= SHORT_ANSWER

def is_same_context(hour1, hour2):
    """
    hour1 must be larger-equal than hour2
    """
    return hour1 - hour2 <= SAME_CONTEXT

def is_same_text(hour1, hour2):
    return hour1 - hour2 <= SAME_TEXT

def is_same_owner(own1, own2):
    """
    return whenever own1 is the same as own2
    """
    return own1==own2

In [62]:
corpus_data = []

with open('corpus.csv', 'w', newline='') as csvfile:
    fieldnames = ['context', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, dialect='unix')
    writer.writeheader()

    last_owner = ''
    last_time = datetime(2000, 1, 1)
    row = {'context':'', 'label':''}
    for i, (owner, time, text) in enumerate(info_chat):
        if is_short(text):                
            if not is_same_owner(last_owner, owner):
                # si no es la misma persona que escribe quiere decir que hubo una respuesta (en este caso corta).
                # el label será la respuesta en sí. Empezará un contexto nuevo de la otra persona.
                row['label'] = text
                writer.writerow(row)
                row['context'] = text
            else:
                # si la misma persona sigue escribiendo
                if is_same_text(time, last_time):
                    # si es parte de la misma conversación, agregamos al contexto
                    row['context'] += ' ' + text
                else:
                    # si "cambia la conversación", podríamos decir que la otra persona no contesto y por ello
                    # etiquetariamos como nonDef
                    row['label'] = 'nonDef'
                    writer.writerow(row)
                    row['context'] = text
        else:
            # si no es una respuesta corta
            if is_same_text(time, last_time):
                row['context'] += ' ' + text
            else:
                row['label'] = 'nonDef'
                writer.writerow(row)
                row['context'] = text

        last_owner = owner
        last_time = time

In [67]:
# Contexto con ventana de 10 conversaciones
FRAME = 10

with open('corpus2.csv', 'w', newline='') as csvfile:
    fieldnames = ['context', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, dialect='unix')
    writer.writeheader()
    
    # No uso el datetime
    # Tampoco tengo en cuenta el usuario que escribe
    row = {'context':[], 'label':''}
    for owner, time, text in info_chat:
        if is_short(text):
            row['label'] = text
            writer.writerow(row)

        # Descartamos la primer conversación en la lista 
        # y agregamos la última 
        if len(row['context']) == FRAME:
            row['context'] = row['context'][1:]
            row['context'].append(text)
        else:
            row['context'].append(text)

In [68]:
from collections import defaultdict

short_answers = {}
short_answers = defaultdict(lambda: 0, short_answers)
with open('corpus2.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        short_answers[row['label']] += 1
        
sorted(short_answers.items(), key=lambda x: x[1], reverse=True)

[('ja', 75),
 ('mati', 32),
 ('dale', 15),
 ('si', 13),
 ('', 7),
 ('de una', 7),
 ('sip', 5),
 ('ja si', 3),
 ('sep', 3),
 ('gracias', 3),
 ('por', 2),
 ('hipótesis', 2),
 ('quizás si', 2),
 ('ahhh', 2),
 ('dale mortal', 2),
 ('dale dale', 2),
 ('joya', 2),
 ('ok genial', 2),
 ('che', 2),
 ('claro', 2),
 ('de uan', 2),
 ('vaiamo', 1),
 ('shhhh', 1),
 ('ja pagas', 1),
 ('concurso', 1),
 ('en agosto', 1),
 ('no ', 1),
 ('este miercoles', 1),
 ('joyaaa', 1),
 ('sumo 😑', 1),
 ('ay', 1),
 ('spoiler no', 1),
 ('httpsyoutubedhyuyg8x31c', 1),
 ('cuando es', 1),
 ('bajón', 1),
 ('casualmente', 1),
 ('y recibirme', 1),
 ('por mail', 1),
 ('😉', 1),
 ('dale ja', 1),
 ('genial', 1),
 ('sw murioo', 1),
 ('viste westworld', 1),
 ('httpswwwfacebookcomevents1307510569364009notiftplanuserinvitednotifid1491686493861284',
  1),
 ('uh', 1),
 ('ja exactamente', 1),
 ('vayamos', 1),
 ('usa adblock', 1),
 ('heroína mujer', 1),
 ('exactament', 1),
 ('mortal¡', 1),
 ('httpsdocsgooglecomdocumentd1jotbxm2g6yte58